#### El cliente nos ha solicitado realizar algunos métodos que resuelvan consultas específicas sobre laS tablaS movies_df, ratings_df y tags_df. Lee cuidadosamente cada consulta y desarrolla el método correspondiente dada la firma del método requerida.

##### Nota: Para poder trabajar con este notebook es necesario haber terminado el ejercicio de la sesión 06

In [1]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
%run utils.ipynb

In [2]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f
import pyspark.sql.types as t

# Creación de sesión de Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ejercicio_7") \
    .getOrCreate()

# Carga de tablas requeridas
root_path = "../resources/data/tmp/parquet/"
names_list = ["06/movies", "06/ratings", "06/tags"]
df_dict = read_tmp_df(names_list)

movies_df = df_dict["06/movies"]
ratings_df = df_dict["06/ratings"]
tags_df = df_dict["06/tags"]

movies_df.show(1, False)
ratings_df.show(1)
tags_df.show(1)

+--------+----------------+-------------------------------------------------+----+
|movie_id|title           |genres                                           |year|
+--------+----------------+-------------------------------------------------+----+
|1       |Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1995|
+--------+----------------+-------------------------------------------------+----+
only showing top 1 row

+-------+--------+------+-------------------+
|user_id|movie_id|rating|               time|
+-------+--------+------+-------------------+
|      1|       1|   4.0|2008-11-03 11:52:19|
+-------+--------+------+-------------------+
only showing top 1 row

+-------+--------+------+-------------------+
|user_id|movie_id|   tag|               time|
+-------+--------+------+-------------------+
| 224183|     832|acting|2017-06-05 07:20:27|
+-------+--------+------+-------------------+
only showing top 1 row



#### Actividad 1:
##### TO DO ->    Para el dataframe "movies_df":
- ##### Genera un método llamado get_all_genres que retorne un DataFrame con únicamente una columna conteniendo todos los valores distintos (sin repetir) de la columna "genres"
    - ##### Firma: def get_all_genres(df: DataFrame) -> DataFrame
    - Apoyate de la funcion explode -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html#pyspark.sql.functions.explode

- ##### Genera un método llamado get_min_year que retorne un valor de tipo int que contenga el menor año registrado (omitiendo nulls)
    - ##### Firma: def get_min_year(df: DataFrame) -> int
    - ##### Necesitarás llamar alguna de las siguientes acciones: take, first, head.

- ##### Genera un método llamado get_min_year que retorne un valor de tipo int que contenga el mayor año registrado (omitiendo nulls)
    - ##### Firma: def get_max_year(df: DataFrame) -> int
    - ##### Necesitarás llamar alguna de las siguientes acciones: take, first, head.

##### NO UTILIZAR withColumn NI withColumnRenamed

In [6]:
## TU CODIGO VA EN ESTA CELDA:

#Importaciones extra 
from pyspark.sql.functions import explode
from pyspark.sql import DataFrame

# Método para obtener todos los géneros únicos (uso de la función explode)
def get_all_genres(df: DataFrame) -> DataFrame:
    distinct_genres = df.selectExpr("explode(genres) as genre").distinct()
    return distinct_genres

# Método para el año mínimo SIN valores nulos
def get_min_year(df: DataFrame) -> int:
    min_year = df.filter(df["year"].isNotNull()).selectExpr("min(year)").first()[0]
    return min_year

# Método para el año máximo SIN valores nulos
def get_max_year(df: DataFrame) -> int:
    max_year = df.filter(df["year"].isNotNull()).selectExpr("max(year)").first()[0]
    return max_year

# Ejecución
all_genres = get_all_genres(movies_df)
min_year = get_min_year(movies_df)
max_year = get_max_year(movies_df)

# Muestra los resultados
all_genres.show()
print("Año mínimo:", min_year)
print("Año máximo:", max_year)


+-----------+
|      genre|
+-----------+
|      Crime|
|    Romance|
|   Thriller|
|  Adventure|
|      Drama|
|        War|
|Documentary|
|    Fantasy|
|    Mystery|
|    Musical|
|  Animation|
|  Film-Noir|
|       IMAX|
|     Horror|
|    Western|
|     Comedy|
|   Children|
|     Action|
|     Sci-Fi|
+-----------+

Año mínimo: 1874
Año máximo: 2023


In [7]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

get_all_genres(movies_df).show(20, False)
"""
Ejemplo de salida esperada (el nombre de la columna podría ser distinto):
+-----------+
|col        |
+-----------+
|Crime      |
|Romance    |
|Thriller   |
|Adventure  |
|Drama      |
|War        |
|Documentary|
|Fantasy    |
|Mystery    |
|Musical    |
|Animation  |
|Film-Noir  |
|IMAX       |
|Horror     |
|Western    |
|Comedy     |
|Children   |
|Action     |
|Sci-Fi     |
+-----------+
"""
print(get_min_year(movies_df))
# Salida esperada: 1874
print(get_max_year(movies_df))
# Salida esperada: 2023

+-----------+
|genre      |
+-----------+
|Crime      |
|Romance    |
|Thriller   |
|Adventure  |
|Drama      |
|War        |
|Documentary|
|Fantasy    |
|Mystery    |
|Musical    |
|Animation  |
|Film-Noir  |
|IMAX       |
|Horror     |
|Western    |
|Comedy     |
|Children   |
|Action     |
|Sci-Fi     |
+-----------+

1874
2023


In [8]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

genres_df = get_all_genres(movies_df)

assert type(genres_df) == DataFrame
assert len(genres_df.columns) == 1

genres_list = genres_df.rdd.map(lambda item: item[0]).collect()
genres_list.sort()
assert genres_list == ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 
                       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
                       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
                       'Thriller', 'War', 'Western']

min_year = get_min_year(movies_df)
assert min_year == 1874

max_year = get_max_year(movies_df)
assert max_year == 2023

#### Actividad 2:
##### TO DO -> Para la tabla "ratings_df" el cliente requiere hacer un análisis de cada "movie_id" para tener una idea de qué tan buena es cada pelicula, asi que nos solicitó desarrollar un método que realice múltiples cálculos.
##### Generar un método que retorne un DataFrame con las columnas especificadas por cada "movie_id":
- ##### Firma: def calculate_rating_values(df: DataFrame) -> DataFrame
- ##### Columnas generadas:
    - ##### nombre: avg_rating, tipo: DoubleType() -> Valor rating promedio (Redondear a 2 decimales)
    - ##### nombre: stddev_rating, tipo: DoubleType() -> Desviacion estándar (stddev_pop) para la columna rating (Redondear a 2 decimales)
        - Para redondear valores utiliza la función round de Spark -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.round.html#pyspark.sql.functions.round
    - ##### nombre: count_rating, tipo: LongType() -> Total de calificaciones recibidas
- ##### ACTUALIZACIÓN: Para aquellas peliculas (movie_id) que no se tenga identificado su año en la tabla "movies_df" nos han solicitado calcular el año de la siguiente manera:
    - ##### nombre: min_time_rating, tipo: TimestampType() -> Fecha más antigua de la columna "time" en la que se asignó el primer rating
##### Nota 1: podemos hacer el calculo de "min_time_rating" en la misma transformación en las que se generan las columnas "avg_rating", "stddev_rating", y "count_rating"
##### Nota 2: Posiblemente requieras analizar las funciones de agregación existentes en Spark -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#aggregate-functions
* El dataframe de salida deberá contar con la siguiente estructura al hacer printSchema():
* |-- movie_id: string
* |-- avg_rating: double
* |-- stddev_rating: double
* |-- count_rating: long
* |-- min_time_rating: timestamp
##### NO UTILIZAR withColumn NI withColumnRenamed

In [228]:
## TU CODIGO VA EN ESTA CELDA:

# Importaciones extra
from pyspark.sql.functions import col, avg, stddev_pop, count, min, round
import pyspark.sql.functions as f
from pyspark.sql.types import DoubleType, LongType, TimestampType
from pyspark.sql import DataFrame

#Importar las librerias con "as f"

# Transformaciones y cálculos
def calculate_rating_values(df: DataFrame) -> DataFrame:
    rating_stats = df.groupBy("movie_id").agg(
        f.round(f.avg("rating"), 2).alias("avg_rating"),
        f.round(stddev_pop("rating"), 2).alias("stddev_rating"),
        f.count("rating").alias("count_rating"),
        f.min("time").cast(TimestampType()).alias("min_time_rating")
    )
    return rating_stats

# Cálculo de los valores
rating_stats = calculate_rating_values(ratings_df)

# Unión del dataframe
result_df = rating_stats.join(movies_df.select("movie_id"), on="movie_id", how="left")

#Impresión de resultados
result_df.show()
result_df.printSchema()


+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|     296|      4.19|         0.95|      108756|1996-02-29 10:48:44|
|  115713|      3.99|         0.83|       21335|2015-01-02 06:05:51|
|    2294|      3.21|         0.96|       14058|1998-10-03 10:54:51|
|   88140|      3.48|         1.01|       14256|2011-07-22 21:23:29|
|    1090|       3.9|         0.84|       20860|1996-09-24 23:05:27|
|   89864|      3.72|         0.83|        6921|2011-09-25 23:29:25|
|  173535|      4.12|         0.97|         125|2017-06-03 06:11:20|
|    2088|      2.57|         1.07|        3982|1998-07-19 16:17:03|
|  102684|      3.13|         1.08|         962|2013-05-24 07:12:47|
|    2136|      2.82|         1.13|        3506|1998-08-01 09:28:36|
|     829|      2.67|         1.18|        2035|1996-07-01 18:09:57|
|    3210|      3.64|         0.93

In [19]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
calculate_rating_values(ratings_df).show(2)
"""
Ejemplo de salida esperada:
+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|     296|      4.19|         0.95|      108756|1996-02-29 10:48:44|
|  115713|      3.99|         0.83|       21335|2015-01-02 06:05:51|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows
"""

+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|     296|      4.19|         0.95|      108756|1996-02-29 10:48:44|
|  115713|      3.99|         0.83|       21335|2015-01-02 06:05:51|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows



'\nEjemplo de salida esperada:\n+--------+----------+-------------+------------+-------------------+\n|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|\n+--------+----------+-------------+------------+-------------------+\n|     296|      4.19|         0.95|      108756|1996-02-29 10:48:44|\n|  115713|      3.99|         0.83|       21335|2015-01-02 06:05:51|\n+--------+----------+-------------+------------+-------------------+\nonly showing top 2 rows\n'

In [20]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

rating_values_df = calculate_rating_values(ratings_df)

assert type(rating_values_df) == DataFrame
assert len(rating_values_df.columns) == 5
assert rating_values_df.count() == 83239

data = rating_values_df \
    .filter(f.col("movie_id") == "296") \
    .collect()[0]
assert data["movie_id"] == "296"
assert data["avg_rating"] == 4.19
assert data["stddev_rating"] == 0.95
assert data["count_rating"] == 108756
assert data["min_time_rating"] == datetime.datetime(1996, 2, 29, 10, 48, 44)

#### Actividad 3:
##### TO DO -> El cliente ha solicitado generar dos tablas con el mismo contenido pero con distinto esquema a partir de la tabla "tags_df", lee a continuación la información enviada con los requerimientos:
- ##### En ambas tablas se requiere agrupar por "movie_id" y calcular:
    - ##### El total de veces en las que aparece cada tag (en mayusculas) .
    - ##### Fecha más antigua en la que se asignó el primer tag.
La primer tabla deberá tener la siguiente estructura:  
*      | movie_id |                 tag_count |        min_time_tag |
*      |        1 | [TERROR : 12, SCI-FI : 2] | 2017-06-02 07:20:27 |
*      |        3 |  [DRAMA : 14, SCI-FI : 4] | 2012-06-02 07:20:27 |
con esquema:
*      |-- movie_id: string
*      |-- tag_count: array
*      |    |-- element: string
*      |-- min_time_tag: timestamp
La segunda tabla deberá tener la siguiente estructura:
*      | movie_id |                   tag_count |        min_time_tag |
*      |        1 | [{TERROR, 12}, {SCI-FI, 2}] | 2017-06-02 07:20:27 |
*      |        3 |  [{DRAMA, 14}, {SCI-FI, 4}] | 2012-06-02 07:20:27 |
con esquema:
*      |-- movie_id: string
*      |-- tag_count: array
*      |    |-- element: struct
*      |    |    |-- tag: string
*      |    |    |-- count: long
*      |-- min_time_tag: timestamp
- #### La generación de la primer tabla es a través del método con la firma:
    - ##### def get_act_3_df1(df: DataFrame) -> DataFrame
- #### La generación de la segunda tabla es a través del método con la firma:
    - ##### def get_act_3_df2(df: DataFrame) -> DataFrame

#### La siguiente información muestra los pasos recomendados para resolver la actividad 3, puedes omitir leer esta parte si asi lo consideras.
##### Para generar ambas tablas primero necesitamos obtener el total veces (count) en las que un "tag" se repite por cada "movie_id", para evitar conteos erroneos hay que convertir cada tag en Mayusculas.
##### El dataframe resultante deberá contener la siguiente estructura: "movie_id", "tag", "count" y "min" donde la columna "count" representa el total de veces que aparecen cada "tag"y "movie_id"; la columna "min" representa el valor mínimo de cada "tag" y "movie_id".
Por ejemplo, dado el dataframe de entrada
*      |user_id|movie_id|   tag|               time|
*      |    183|     100|sci-fi|2012-06-02 07:20:27|
*      |     12|     832| Drama|2017-06-01 07:20:27|
*      |    251|     100|SCI-FI|2009-06-04 07:20:27|
*      |    265|     832| DRAMA|2015-06-08 07:20:27|
*      |     22|     100|terror|2020-06-06 07:20:27|

debemos obtener el siguiente dataframe (el nombre de columnas "count" y "min" podria ser distinto):

*      | movie_id |    tag | count |                 min |
*      |      100 | TERROR |     1 | 2020-06-06 07:20:27 |
*      |      100 | SCI-FI |     2 | 2009-06-04 07:20:27 |
*      |      832 |  DRAMA |     2 | 2015-06-08 07:20:27 |

##### Para generar la primer tabla necesitamos concatenar las columnas "tag" y "count", posteriormente agrupando por "movie_id" generaremos la lista de elementos requerida. La columna "min_time_tag" representa el valor mínimo de cada "movie_id".
- ##### Funciones de Spark recomendadas:
    - upper -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.upper.html#pyspark.sql.functions.upper
    - concat_ws -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.concat_ws.html#pyspark.sql.functions.concat_ws
    - concat -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.concat.html#pyspark.sql.functions.concat
    - collect_list -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.collect_list.html#pyspark.sql.functions.collect_list
    - collect_set -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.collect_set.html#pyspark.sql.functions.collect_set
##### La primer tabla quedará estructurada de la siguiente manera, donde la columna "tag_count" es de tipo ArrayType(StringType()) y la columna "min_time_tag" es de tipo TimestampType():
Dado el dataframe de entrada
*      | movie_id |    tag | count |                 min |
*      |        1 | TERROR |    12 | 2020-06-06 07:20:27 |
*      |        1 | SCI-FI |     2 | 2015-06-06 07:20:27 |
*      |        3 |  DRAMA |    14 | 2004-06-06 07:20:27 |
*      |        3 | SCI-FI |     4 | 2012-06-06 07:20:27 |
debemos obtener el siguiente dataframe
*       | movie_id |                 tag_count |        min_time_tag |
*       |        1 | [TERROR : 12, SCI-FI : 2] | 2015-06-06 07:20:27 |
*       |        3 |  [DRAMA : 14, SCI-FI : 4] | 2004-06-06 07:20:27 |
##### Para generar la segunda tabla necesitamos agregar en una estructura las columnas "tag" y "count", posteriormente agrupando por movie_id generaremos la lista de elementos requerida. La columna "min_time_tag" representa el valor mínimo de cada "movie_id".
- ##### Funciones de Spark recomendadas:
    - upper -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.upper.html#pyspark.sql.functions.upper
    - struct -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.struct.html#pyspark.sql.functions.struct
    - collect_list -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.collect_list.html#pyspark.sql.functions.collect_list
    - collect_set -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.collect_set.html#pyspark.sql.functions.collect_set
##### La segunda tabla quedará estructurada de la siguiente manera, donde la columna "tag_count" de tipo ArrayType(StructType(StringType(),LongType()))  y la columna "min_time_tag" es de tipo TimestampType():
Dado el dataframe de entrada
*      | movie_id |    tag | count |                 min |
*      |        1 | TERROR |    12 | 2020-06-06 07:20:27 |
*      |        1 | SCI-FI |     2 | 2015-06-06 07:20:27 |
*      |        3 |  DRAMA |    14 | 2004-06-06 07:20:27 |
*      |        3 | SCI-FI |     4 | 2012-06-06 07:20:27 |
debemos obtener el siguiente dataframe
*      | movie_id |                   tag_count |        min_time_tag |
*      |        1 | [{TERROR, 12}, {SCI-FI, 2}] | 2015-06-06 07:20:27 |
*      |        3 |  [{DRAMA, 14}, {SCI-FI, 4}] | 2004-06-06 07:20:27 |

In [78]:
## TU CODIGO VA EN ESTA CELDA, PUEDES GENERAR MÉTODOS O VARIABLES NUEVAS SI ASI LO REQUIERES
# Importaciones
from pyspark.sql import DataFrame
from pyspark.sql.functions import upper, count, min, struct, collect_list, concat_ws
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, LongType, TimestampType

# Convertir "tag" a mayúsculas
tags_df = tags_df.select(upper("tag").alias("tag"), "movie_id", "time")

# Calcular el conteo y la fecha mínima por "movie_id" y "tag"
tags_grouped = tags_df.groupBy("movie_id", "tag").agg(count("*").alias("count"), min("time").alias("min_time_tag"))

# Generar la primera tabla
def get_act_3_df1(df):
    tag_count_expr = concat_ws(" : ", df["tag"], df["count"])
    table1 = df.groupBy("movie_id") \
                .agg(collect_list(tag_count_expr).alias("tag_count"), min("min_time_tag").alias("min_time_tag"))
    return table1

# Generar la segunda tabla
def get_act_3_df2(df):
    tag_count_struct = struct(df["tag"].alias("tag"), df["count"].alias("count"))
    table2 = df.groupBy("movie_id") \
                .agg(collect_list(tag_count_struct).alias("tag_count"), min("min_time_tag").alias("min_time_tag"))
    return table2

# Obtener las tablas
act_3_df1 = get_act_3_df1(tags_grouped)
act_3_df2 = get_act_3_df2(tags_grouped)

# Mostrar resultados
act_3_df1.show(truncate=False)
act_3_df2.show(truncate=False)

# Imprimir esquema
act_3_df1.printSchema()
act_3_df2.printSchema()

+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [79]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
act_3_df1.show(2, False)
act_3_df2.show(2, False)
"""
Ejemplo de salida esperada:
+--------+------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                         |min_time_rating    |
+--------+------------------------------------------------------------------+-------------------+
|100062  |[PRESS-GANGED : 1, WORLD WAR II : 1, WAR : 1, FATE : 1]           |2018-05-26 16:40:54|
|100070  |[GOOD HUMOUR : 1, STRUGGLING CAREER : 1, COMEDY : 1, COMEDIAN : 2]|2017-05-19 17:17:36|
+--------+------------------------------------------------------------------+-------------------+
only showing top 2 rows

+--------+----------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                             |min_time_rating    |
+--------+----------------------------------------------------------------------+-------------------+
|100062  |[{PRESS-GANGED, 1}, {WORLD WAR II, 1}, {WAR, 1}, {FATE, 1}]           |2018-05-26 16:40:54|
|100070  |[{GOOD HUMOUR, 1}, {STRUGGLING CAREER, 1}, {COMEDY, 1}, {COMEDIAN, 2}]|2017-05-19 17:17:36|
+--------+----------------------------------------------------------------------+-------------------+
only showing top 2 rows
"""

+--------+--------------------------------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                                                   |min_time_tag       |
+--------+--------------------------------------------------------------------------------------------+-------------------+
|100070  |[GOOD HUMOUR : 1, STRUGGLING CAREER : 1, COMEDY : 1, COMEDIAN : 2]                          |2017-05-19 17:17:36|
|100277  |[SILENT FILM : 1, MIGUEL GOMES : 1, ADULTERY : 1, AFRICA : 1, PORTUGAL : 1, TRAGIC LOVE : 1]|2013-02-04 02:24:23|
+--------+--------------------------------------------------------------------------------------------+-------------------+
only showing top 2 rows

+--------+--------------------------------------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                                                  

'\nEjemplo de salida esperada:\n+--------+------------------------------------------------------------------+-------------------+\n|movie_id|tag_count                                                         |min_time_rating    |\n+--------+------------------------------------------------------------------+-------------------+\n|100062  |[PRESS-GANGED : 1, WORLD WAR II : 1, WAR : 1, FATE : 1]           |2018-05-26 16:40:54|\n|100070  |[GOOD HUMOUR : 1, STRUGGLING CAREER : 1, COMEDY : 1, COMEDIAN : 2]|2017-05-19 17:17:36|\n+--------+------------------------------------------------------------------+-------------------+\nonly showing top 2 rows\n\n+--------+----------------------------------------------------------------------+-------------------+\n|movie_id|tag_count                                                             |min_time_rating    |\n+--------+----------------------------------------------------------------------+-------------------+\n|100062  |[{PRESS-GANGED, 1}, {WORLD W

In [80]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
assert len(act_3_df1.columns) == 3
assert "movie_id" in act_3_df1.columns
assert "tag_count" in act_3_df1.columns
assert "min_time_tag" in act_3_df1.columns
assert schema_to_ddl(act_3_df1.select("movie_id", "tag_count", "min_time_tag")) == 'movie_id STRING,tag_count ARRAY<STRING> NOT NULL,min_time_tag TIMESTAMP'
assert act_3_df1.count() == 53452

assert len(act_3_df2.columns) == 3
assert "movie_id" in act_3_df2.columns
assert "tag_count" in act_3_df2.columns
assert "min_time_tag" in act_3_df2.columns
assert schema_to_ddl(act_3_df2.select("movie_id", "tag_count", "min_time_tag")) == 'movie_id STRING,tag_count ARRAY<STRUCT<tag: STRING, count: BIGINT>> NOT NULL,min_time_tag TIMESTAMP'
assert act_3_df2.count() == 53452

#### Actividad 4:
##### TO DO -> El cliente ha solicitado que resolvamos dos consultas con la salida de la actividad 3, como no especificó con cuál dataframe requiere la consulta lo realizaremos con ambos dataframes (act_3_df1 y act_3_df2).
##### Las consultas a resolver son:
- ##### 1.- ¿Cuál es la pelicula (movie_id) con más tags con el valor "SCI-FI"?
    - ##### Estrictamente especificó no tomar en cuenta datos como "REALISTIC SCI-FI", "HARD SCI-FI", etc.
- ##### 2.- ¿Cuántas peliculas fueron etiquetadas como "SCI-FI"?
##### La forma de resolver estas consultas será a través de un método el cual va a retornar una tupla (str, int), donde el primer elemento (str) representa el resultado de la consulta 1 y el segundo elemento (int) representa el resultado de la consulta 2
- ##### La firma del método que utilizará cada dataFrame es:
    - ##### def exercise_4_df1(df: DataFrame) -> firma para el dataFrame act_3_df1
    - ##### def exercise_4_df2(df: DataFrame) -> firma para el dataFrame act_3_df2
##### Funciones de Spark recomendadas:
- explode -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html#pyspark.sql.functions.explode
- regexp_extract -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.regexp_extract.html#pyspark.sql.functions.regexp_extract
##### Funciones de la clase Column recomendadas:
- like -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.like.html#pyspark.sql.Column.like
- ilike -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.ilike.html#pyspark.sql.Column.ilike
- getField -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.getField.html#pyspark.sql.Column.getField
##### Acciones que podrias utilizar:
- count, first, head, take

In [224]:
## TU CODIGO VA EN ESTA CELDA, PUEDES GENERAR MÉTODOS O VARIABLES SI ASI LO REQUIERES
# Importaciones
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, count, coalesce, explode, regexp_extract

#CONSULTA 1 - PRIMER DF
def exercise_4_df1(df: DataFrame):
    var1 = act_3_df1.select(
        "movie_id", explode(col("tag_count")).alias("genre")
        ).filter(
            col("genre").like('SCI-FI :%')
                )

    new_var = var1.select(
        "movie_id", "genre", regexp_extract(col("genre"),"\d+", 0).cast(IntegerType()).alias("coincidence")
                ).orderBy(col("coincidence").desc()).select("movie_id").first()
    # print(new_var[0])

    return (new_var[0], var1.count())


#CONSULTA 2 - PRIMER DF
    # var1.count()
    # print(var1.count())

def exercise_4_df2(df: DataFrame):
# CONSULTA 1 - SEGUNDO DF
    var2 = act_3_df2.select(
        "movie_id", explode(col("tag_count")).alias("genre")
                    ).filter(
                        col("genre").getField("tag").like('SCI-FI')
                            )

    new_var2 = var2.select(
        "movie_id", col("genre").getField("count").alias('new_genre')
                            ).orderBy(col("new_genre").desc()).select("movie_id").first()

    # print(new_var2[0])

#CONSULTA 2 - SEGUNDO DF
    # var2.count()
    # print(var2.count())

    return (new_var2[0], var2.count())

In [225]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
print(exercise_4_df1(act_3_df1))
print(exercise_4_df2(act_3_df2))
# Salida esperada en ambos casos
#('260', 854)

('260', 854)
('260', 854)


In [226]:
movie_id, count = exercise_4_df1(act_3_df1)
assert movie_id == "260"
assert count == 854

movie_id, count = exercise_4_df2(act_3_df2)
assert movie_id == "260"
assert count == 854

In [229]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(movies_df, "07/movies"),
       (act_3_df1, "07/tags_p1"),
       (act_3_df2, "07/tags_p2"),
       (calculate_rating_values(ratings_df), "07/ratings")]

write_tmp_df(dfs)